# Probablistic Model

In [1]:
import math
import torch
import numpy as np
import pandas as pd
import tensorflow as tf
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from sklearn import preprocessing
from Training import model, utils, dataset, train
from sklearn.model_selection import train_test_split

import keras.backend as K

In [2]:
def check_acc(y_hat,y,margin=0.05):
    a_err = (np.abs(y_hat - y)) # get normalized error 
    err = np.divide(a_err, y, out=a_err, where=y==0)
    assert(err.shape == y.shape)
    num_correct = 0
    for row in err:
        num_in_row = len(np.where(row < margin)[0]) # margin * 100 because 
        if num_in_row == len(row):
            num_correct += 1

    num_samples = y.shape[0]
    correct_idx = np.where(err < margin)
    num_part_correct = len(correct_idx[0])
    num_part_samples = y.shape[0] * y.shape[1]
    print(f"Correct = {num_correct} / {num_samples}")
    return (num_correct/num_samples)

#### Load Data

In [3]:
data = utils.parseGainAndBWCsv2("Data/BW-3000.csv").astype(float)

In [4]:
print(data[0])

[6.200e+02 2.880e-06 3.775e+00 8.280e+10]


##### Split data into inputs X and target Y Normalize to [0,1] (inclusive) and create Training and Testign Splits


In [5]:
data = preprocessing.MinMaxScaler((0,1)).fit_transform(data)
X = data[:,:2]
Y = data[:,2:]
X_train, X_test, y_train, y_test = train_test_split(X, Y)

#### Define Model and Parameters

##### Define model

* Model: MLP with variable hidden layer number and width  
* Output layer: 
    * Node 1 and 2 are the mean for probability mean 
    * Node 3 and 4 are the diagonal of the covariance matrix


In [6]:
# num_hidden_layers = 5
# hidden_layer_size = 500

# model = tf.keras.models.Sequential([])

# for i in range(num_hidden_layers):
#     model.add(tf.keras.layers.Dense(hidden_layer_size, activation='relu'))
# model.add(tf.keras.layers.Dense(4, activation='sigmoid')) # add output layer: node 1,2 are 

In [7]:
test_model = model.Sigmoid_Net(2,4)

##### Define Loss

* Loss: guassian_nll
    * cite: https://gist.github.com/sergeyprokudin/4a50bf9b75e0559c1fcd2cae860b879e

In [8]:
optimizer = optim.Adam(test_model.parameters(),lr=0.01)
loss_fn = nn.GaussianNLLLoss()

dataset1 = dataset.CircuitSynthesisGainAndBandwidthManually(Y, X)
train_dataset, val_dataset = utils.splitDataset(dataset1, 0.8)
    
train_data = DataLoader(train_dataset,batch_size = 500)
validation_data = DataLoader(val_dataset, batch_size = 500)

In [9]:
epochs = 1000
losses, accs, part_accs = train.trainProbModel(test_model, train_data, loss_fn, optimizer, num_epochs=epochs, print_every=10)

t = 10, loss = -0.7405
t = 20, loss = -0.7426
t = 30, loss = -0.7426
t = 40, loss = -0.7426
t = 50, loss = -0.7426
t = 60, loss = -0.7426
t = 70, loss = -0.7426
t = 80, loss = -0.7426
t = 90, loss = -0.7426
t = 100, loss = -0.7426
t = 110, loss = -0.7426
t = 120, loss = -0.7426
t = 130, loss = -0.7426
t = 140, loss = -0.7426
t = 150, loss = -0.7426
t = 160, loss = -0.8252
t = 170, loss = -0.9308
t = 180, loss = -0.9431
t = 190, loss = -1.0860
t = 200, loss = -1.1366
t = 210, loss = -1.1588
t = 220, loss = -1.1595
t = 230, loss = -1.0536
t = 240, loss = -1.1294
t = 250, loss = -1.1487
t = 260, loss = -1.1533
t = 270, loss = -1.1561
t = 280, loss = -1.1574
t = 290, loss = -1.1585
t = 300, loss = -1.1596
t = 310, loss = -1.1603
t = 320, loss = -1.1611
t = 330, loss = -1.1619
t = 340, loss = -1.1473
t = 350, loss = -1.1529
t = 360, loss = -1.1590
t = 370, loss = -1.1549
t = 380, loss = -1.1590
t = 390, loss = -1.1576
t = 400, loss = -1.1588
t = 410, loss = -1.1590
t = 420, loss = -1.1590
t

KeyboardInterrupt: 

## Testing scipt

My idea of the testing script. 
It takes the inverse simulator prediction (Y -> X^)  
Pass X^ to mock_simulator to get Y^   
evaluate performance from Y to Y^  
I don't know if the data is being shuffled at all could contribute too the poor performance

In [10]:
x_preds = test_model(torch.Tensor(Y))
#print(x_preds)
mock_simulator = tf.keras.models.load_model('mock_simulator')

final_preds = []
for i in range(x_preds.shape[0]):
    means = x_preds.detach().numpy()[i,:2].T
    var = x_preds.detach().numpy()[i,2:].T

    varM = np.zeros((2,2))
    np.fill_diagonal(varM,var)

    pts = np.random.default_rng().multivariate_normal(means, varM, 1).T # get predicted X value 
    y_hat = mock_simulator(pts.T) # feed back through simulator to get evaluated performance
    final_preds.append(y_hat)
final_preds = np.array(final_preds).reshape(-1,2)
    

# for i,d in enumerate(final_preds):
#     print(Y[i],d)
print(final_preds)


2022-04-29 06:13:22.251016: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-04-29 06:13:22.251066: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-29 06:13:22.251097: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (jupyter-reevest): /proc/driver/nvidia/version does not exist
2022-04-29 06:13:22.251430: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


[[0.52669483 0.67764723]
 [0.5254123  0.6803887 ]
 [0.5595935  0.6230493 ]
 ...
 [0.         1.6541357 ]
 [0.         0.95250136]
 [0.17390898 0.6078639 ]]


In [11]:
check_acc(Y,final_preds,margin=.05)

Correct = 743 / 3022


/tmp/ipykernel_30399/4237178316.py:3: RuntimeWarning: divide by zero encountered in true_divide
  err = np.divide(a_err, y, out=a_err, where=y==0)
/tmp/ipykernel_30399/4237178316.py:3: RuntimeWarning: invalid value encountered in true_divide
  err = np.divide(a_err, y, out=a_err, where=y==0)


0.24586366644606222

In [12]:
def gaussian_nll(ytrue, ypreds):
    """Keras implmementation of multivariate Gaussian negative loglikelihood loss function. 
    This implementation implies diagonal covariance matrix.
    
    Parameters
    ----------
    ytrue: tf.tensor of shape [n_samples, n_dims]
        ground truth values
    ypreds: tf.tensor of shape [n_samples, n_dims*2]
        predicted mu and logsigma values (e.g. by your neural network)
        
    Returns
    -------
    neg_log_likelihood: float
        negative loglikelihood averaged over samples
        
    This loss can then be used as a target loss for any keras model, e.g.:
        model.compile(loss=gaussian_nll, optimizer='Adam') 
    
    """
    print(f"ytrue: {ytrue.shape}")
    print(f"ypreds: {ypreds.shape}")
    n_dims = int(int(ypreds.shape[1])/2)
    mu = ypreds[:, 0:n_dims]
    logsigma = ypreds[:, n_dims:]
    sigma = ypreds[:, n_dims:]
    
    if K.any(sigma < 0):
        print(sigma)
        raise ValueError("sigma has negative entry/entries")
        
    #mse = -0.5*K.sum(K.square((ytrue-mu)/K.exp(logsigma)),axis=1)
    #sigma_trace = -K.sum(logsigma, axis=1)
    #log2pi = -0.5*n_dims*np.log(2*np.pi)
    
    log_likelihood = 0.5 * K.log(sigma**2) + (ytrue-mu)**2 / (sigma)

    return K.mean(log_likelihood)

In [ ]:
# loss_fn = gaussian_nll

# optim = tf.keras.optimizers.Adam(
#     learning_rate=0.001)
# model.compile(optimizer=optim,
#               loss=loss_fn)

In [ ]:
# model.fit(X_train, y_train, epochs=1000, batch_size= 700, validation_data=(X_test, y_test))